In [83]:
import os
import boto3
from pathlib import Path

AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY', '005dc3bfa158d830000000002')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY', 'K005NTUEFXRIi2mMPBKFj0EFWbXMGmA')
AWS_ENDPOINT_URL = os.getenv('AWS_ENDPOINT_URL', 's3.us-east-005.backblazeb2.com')

class S3Client:
    def __init__(
        self,
        default_bucket_name="smartllm",
        endpoint_url=AWS_ENDPOINT_URL,
        access_key=AWS_ACCESS_KEY_ID,
        secret_key=AWS_SECRET_ACCESS_KEY,
    ) -> None:
        self.default_bucket = default_bucket_name
        self.endpoint_url = endpoint_url
        self.https_endpoint_url = f"https://{endpoint_url}" if not endpoint_url.startswith('https://') else endpoint_url
        self.client = boto3.client(
            's3',
            endpoint_url=self.https_endpoint_url,
            aws_access_key_id=access_key,
            aws_secret_access_key=secret_key,
        )
        
    def create_bucket(self, bucket_name: str = None) -> dict:
        bucket_name = bucket_name or self.default_bucket
        response = self.client.create_bucket(Bucket=bucket_name)
        return response
    
    def list_buckets(self) -> list[dict]:
        response = self.client.list_buckets()
        return response.get('Buckets', [])

    def list_objects(self, bucket_name: str = None) -> list[dict]:
        bucket_name = bucket_name or self.default_bucket
        response = self.client.list_objects(Bucket=bucket_name)
        return response.get('Contents', [])
    
    def download_file(self, object_key: str, local_parent: str = None, bucket_name: str = None) -> dict:
        bucket_name = bucket_name or self.default_bucket
        local_parent: Path = Path(local_parent) if local_parent else Path('.')
        local_path = local_parent/object_key or Path(object_key)
        local_path.parent.mkdir(parents=True, exist_ok=True)
        self.client.download_file(bucket_name, object_key, local_path.as_posix())
        return local_path
    
    def upload_file(self, local_path: str, object_key: str = None, bucket_name: str = None) -> dict:
        bucket_name = bucket_name or self.default_bucket
        local_path: Path = Path(local_path)
        if not local_path.exists():
            raise FileNotFoundError(f"File not found: {local_path}")
        object_key = local_path.as_posix() if not object_key else object_key
        self.client.upload_file(local_path.as_posix(), bucket_name, object_key)
        return f"https://{bucket_name}.{self.endpoint_url}/{object_key}"
    
    def delete_file(self, object_key: str, bucket_name: str = None) -> dict:
        bucket_name = bucket_name or self.default_bucket
        response = self.client.delete_object(Bucket=bucket_name, Key=object_key)
        return response
    
    def get_presigned_url(self, object_key: str, bucket_name: str = None, expiration: int = 3600) -> str:
        bucket_name = bucket_name or self.default_bucket
        response = self.client.generate_presigned_url(
            'get_object',
            Params={
                'Bucket': bucket_name,
                'Key': object_key
            },
            ExpiresIn=expiration,
            HttpMethod='GET',
        )
        return response

s3 = S3Client()

In [84]:
s3.upload_file('requirements.txt')

'https://smartllm.s3.us-east-005.backblazeb2.com/requirements.txt'

In [27]:
s3.download_file('smartllm', 'requirements.png')

PosixPath('requirements.png')

In [85]:
s3.list_objects("smartllm")

[{'Key': 'JupySQL.ipynb',
  'LastModified': datetime.datetime(2024, 3, 20, 11, 48, 42, 565000, tzinfo=tzutc()),
  'ETag': '"de32a7b6ba8bb9489c83e3ad6fe4b618"',
  'Size': 22042,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': '', 'ID': 'dc3bfa158d83'}},
 {'Key': 'requirements.txt',
  'LastModified': datetime.datetime(2024, 3, 20, 13, 13, 12, 326000, tzinfo=tzutc()),
  'ETag': '"99b04e6f345a8045aaaf96e7b6a595b9"',
  'Size': 4644,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': '', 'ID': 'dc3bfa158d83'}},
 {'Key': 'tasks.json',
  'LastModified': datetime.datetime(2024, 3, 20, 12, 2, 32, 458000, tzinfo=tzutc()),
  'ETag': '"a88780309834231119d31f8c205c81ed"',
  'Size': 547,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': '', 'ID': 'dc3bfa158d83'}},
 {'Key': 'vscode/tasks.json',
  'LastModified': datetime.datetime(2024, 3, 20, 12, 2, 10, 741000, tzinfo=tzutc()),
  'ETag': '"a88780309834231119d31f8c205c81ed"',
  'Size': 547,
  'StorageClass': 'STANDARD',
  'Owner':